<a href="https://colab.research.google.com/github/csrajath/CSC_8980_NLP/blob/main/csc_8980_NLP_exam1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name: **Rajath Chikkatur Srinivasa** \
PantherID: **002552425**

In [6]:
# importing required libraries
import re, random, os, nltk,spacy
from collections import Counter, defaultdict
import pandas as pd
import numpy as np
from nltk import bigrams, trigrams
# all sklearn imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import *
# importing models
#importing classifier types
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.pipeline import make_pipeline



**Question 1) (20 points) Write a generic function that takes: Classification algorithm name, vectorization method name, training set with labels as parameters (total of 3 parameters should be passed). The function should take the classification algorithm name, the vectorization method’s name, and the training set and train the desired model. Use the default training parameters for the models we have seen in class. This function should return the trained model**

**Question 2) (30 points) Using the function from question 1 to build the following models:
a) Model a: Naive Bayes, Vectorizer: TFIDF and Bag of Words, Training set should be 75% of the provided dataset. Leaving the remaining 25% for testing.
b) Model b: RandomForest, Vectorizer: TFIDF and Bag of Words, Training set should be 70% of the provided dataset. Leaving the remaining 30% for testing.
c) Model c: Support Vector Machines (SVC in sklearn), Vectorizer: TFIDF and Bag of Words, Training set should be 60% of the provided dataset. Leaving the remaining 40% for testing. NOTE: Set the random seed to: 12345. This needs to be consistently set to train the model AND split the data in test and train. If this is not done correctly, you will lose points as your answers will not be comparable with the grading key**